In [1]:
from neurokit2.data import read_acqknowledge

In [2]:
datapath = './pilot2/sangsu/sangsu_VR2024-11-25T17_13_04.acq'
data = read_acqknowledge(datapath)

In [3]:
data

(         PPG, X, PPGED-R  RSP, X, RSPEC-R  EDA, Y, PPGED-R
 0               0.417786            -10.0         8.897400
 1               0.419006            -10.0         8.895874
 2               0.421448            -10.0         8.895874
 3               0.422668            -10.0         8.894348
 4               0.423279            -10.0         8.897400
 ...                  ...              ...              ...
 3624728         0.111694            -10.0        18.019104
 3624729         0.110779            -10.0        18.022156
 3624730         0.111389            -10.0        18.019104
 3624731         0.110168            -10.0        18.020630
 3624732         0.107117            -10.0        18.019104
 
 [3624733 rows x 3 columns],
 2000.0)

In [4]:
import bioread
data = bioread.read_file(datapath)
for m in data.event_markers:
    print('{0}: Channel {1}, type {2}'.format(m.text, m.channel_name, m.type))

Segment 1: Channel None, type Append
1,Start: Channel None, type Stimulus Delivery
1,StartP: Channel None, type Stimulus Delivery
1,End,1732522961610.36: Channel None, type Stimulus Delivery
2,Start: Channel None, type Stimulus Delivery
2,StartP: Channel None, type Stimulus Delivery
2,End,1732523016665.66: Channel None, type Stimulus Delivery
3,Start: Channel None, type Stimulus Delivery
3,StartP: Channel None, type Stimulus Delivery
3,End,1732523215179.86: Channel None, type Stimulus Delivery
4,Start: Channel None, type Stimulus Delivery
4,StartP: Channel None, type Stimulus Delivery
4,End,1732523543870.81: Channel None, type Stimulus Delivery
5,Start: Channel None, type Stimulus Delivery
5,StartP: Channel None, type Stimulus Delivery
5,End,1732523714881.32: Channel None, type Stimulus Delivery
6,Start: Channel None, type Stimulus Delivery
6,StartP: Channel None, type Stimulus Delivery
6,End,1732524033745.38: Channel None, type Stimulus Delivery
7,Start: Channel None, type Stimulus De

In [92]:
import os
from collections import Counter
import numpy as np
import pandas as pd
from neurokit2.signal import signal_resample


def read_acqknowledge_with_markers(filename, sampling_rate="max", resample_method="interpolation", impute_missing=True):
    """
    Read and format a BIOPAC's AcqKnowledge file into a pandas' dataframe, including event markers.

    Parameters
    ----------
    filename : str
        Filename (with or without the extension) of a BIOPAC's AcqKnowledge file (e.g., "data.acq").
    sampling_rate : int or "max"
        Desired sampling rate in Hz. "max" uses the maximum recorded sampling rate.
    resample_method : str
        Method of resampling.
    impute_missing : bool
        Whether to impute missing values in the signal.

    Returns
    ----------
    df : DataFrame
        The AcqKnowledge file as a pandas dataframe.
    event_markers : DataFrame
        Event markers with columns ['Time (s)', 'Channel', 'Type', 'Text'].
    sampling_rate : int
        Sampling rate used in the data.

    """
    try:
        import bioread
    except ImportError:
        raise ImportError("Please install the 'bioread' module (`pip install bioread`).")

    # Check filename
    if not filename.endswith(".acq"):
        filename += ".acq"

    if not os.path.exists(filename):
        raise ValueError(f"File not found: {filename}")

    # Read the AcqKnowledge file
    file = bioread.read_file(filename)

    # Determine sampling rate
    if sampling_rate == "max":
        sampling_rate = max(channel.samples_per_second for channel in file.channels)

    # Process data channels
    data = {}
    channel_counter = Counter()
    for channel in file.channels:
        signal = np.array(channel.data)

        # Handle missing data
        if impute_missing and np.isnan(signal).any():
            signal = pd.Series(signal).fillna(method="pad").values

        # Resample signal
        if channel.samples_per_second != sampling_rate:
            signal = signal_resample(
                signal,
                sampling_rate=channel.samples_per_second,
                desired_sampling_rate=sampling_rate,
                method=resample_method,
            )

        # Handle duplicate channel names
        channel_name = channel.name
        if channel_counter[channel_name] > 0:
            channel_name = f"{channel_name} ({channel_counter[channel_name]})"
        data[channel_name] = signal
        channel_counter[channel.name] += 1

    # Align signal lengths
    max_length = max(len(signal) for signal in data.values())
    for channel_name, signal in data.items():
        if len(signal) < max_length:
            data[channel_name] = np.pad(signal, (0, max_length - len(signal)), mode="edge")

    # Create DataFrame for signal data
    df = pd.DataFrame(data)

    # Extract event markers
    event_markers = []
    for marker in file.event_markers:
        event_markers.append({
            "Time (s)": marker.sample_index / sampling_rate,
            "Channel": marker.channel_name,
            "Type": marker.type,
            "Text": marker.text
        })
    event_markers_df = pd.DataFrame(event_markers)
    event_markers_df['Sample'] = event_markers_df['Time (s)']*2000
    event_markers_df[['scene', 'marker']] = event_markers_df['Text'].str.split(',', n=2, expand=True)[[0, 1]]
    event_markers_df = event_markers_df[event_markers_df['Type'] != 'Append']
    event_markers_df.drop(columns=['Text','Channel','Type','Time (s)'], inplace=True)
    event_markers_df['scene'] = pd.to_numeric(event_markers_df['scene'])
    event_markers_df['Sample'] = event_markers_df['Sample'].apply(np.int64)
    event_markers_df.set_index('Sample', inplace=True)
    result = pd.concat([df, event_markers_df], axis=1)
    
    ############ Time에 sampling_rate 곱해서 frame 단위로 바꾸고, Time 기본은 남겨두고 frame을 index로 할 것.

    return df, event_markers_df, sampling_rate, result


In [93]:
df, event_markers_df, sampling_rate, result = read_acqknowledge_with_markers(datapath)

In [82]:
result

,"PPG, X, PPGED-R","RSP, X, RSPEC-R","EDA, Y, PPGED-R",scene,marker
0,0.417786,-10.0,8.897400,NaN,NaN
1,0.419006,-10.0,8.895874,NaN,NaN
2,0.421448,-10.0,8.895874,NaN,NaN
3,0.422668,-10.0,8.894348,NaN,NaN
4,0.423279,-10.0,8.897400,NaN,NaN
...,...,...,...,...,...
3624728,0.111694,-10.0,18.019104,NaN,NaN
3624729,0.110779,-10.0,18.022156,NaN,NaN
3624730,0.111389,-10.0,18.019104,NaN,NaN
3624731,0.110168,-10.0,18.020630,NaN,NaN


In [71]:
event_markers_df['Sample'] = event_markers_df['Sample'].apply(np.int64)
event_markers_df.set_index('Sample', inplace=True)


In [73]:
event_markers_df
result = pd.concat([df, event_markers_df], axis=1)

In [34]:
_df = event_markers_df
_df['Sample'] = _df['Time (s)']*2000
_df[['scene', 'marker']] = _df['Text'].str.split(',', n=2, expand=True)[[0, 1]]
_df = _df[_df['Type'] != 'Append']
_df.drop(columns=['Text','Channel','Type','Time (s)'], inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_8260\292428174.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df.drop(columns=['Text','Channel','Type','Time (s)'], inplace=True)


In [ ]:
for i in range(1,8):

    startidx = result[(result['scene']==i)&(result['marker']=="Start")].index[0]
    startpidx = result[(result['scene']==i)&(result['marker']=="StartP")].index[0]
    endidx = result[(result['scene']==i)&(result['marker']=="End")].index[0]
    result.loc[startidx:endidx, 'scene'] = i
    result.loc[startidx:startpidx, 'marker'] = "Preparing"
    result.loc[startpidx:endidx, 'marker'] = "Ongoing"



In [117]:
result.iloc[startidx:endidx+1]

,"PPG, X, PPGED-R","RSP, X, RSPEC-R","EDA, Y, PPGED-R",scene,marker
1056624,-0.572510,-10.000000,16.894531,1.0,Preparing
1056625,-0.575562,-10.000000,16.896057,1.0,Preparing
1056626,-0.577087,-10.000000,16.896057,1.0,Preparing
1056627,-0.577393,-10.000000,16.899109,1.0,Preparing
1056628,-0.580139,-10.000000,16.894531,1.0,Preparing
...,...,...,...,...,...
1154766,0.288391,-9.781189,15.686035,1.0,Ongoing
1154767,0.295410,-9.781189,15.687561,1.0,Ongoing
1154768,0.302429,-9.780579,15.687561,1.0,Ongoing
1154769,0.310059,-9.780884,15.692139,1.0,Ongoing


In [10]:
event_markers_df

,Time (s),Channel,Type,Text
0,0.000,None,Append,Segment 1
1,528.312,None,Stimulus Delivery,"1,Start"
2,528.503,None,Stimulus Delivery,"1,StartP"
3,577.385,None,Stimulus Delivery,"1,End,1732522961610.36"
4,577.875,None,Stimulus Delivery,"2,Start"
5,578.334,None,Stimulus Delivery,"2,StartP"
6,632.432,None,Stimulus Delivery,"2,End,1732523016665.66"
7,632.921,None,Stimulus Delivery,"3,Start"
8,633.240,None,Stimulus Delivery,"3,StartP"
9,830.946,None,Stimulus Delivery,"3,End,1732523215179.86"
